# 🎓 Predicción de Deserción Estudiantil — Minería de Datos
## Metodología CRISP-DM Completa (6 Fases)

---

| Dato | Valor |
|------|-------|
| **Carrera** | Ciencia de Datos e Inteligencia Artificial |
| **Facultad** | Ciencias Matemáticas y Físicas |
| **Metodología** | CRISP-DM |
| **Dataset** | Record estudiantil anonimizado |
| **Entrada** | Archivo Excel (.xlsx) |
| **Salida** | Modelo entrenado + artefactos para Streamlit |

### Estructura del Notebook
1. **Fase 1** — Comprensión del Negocio
2. **Fase 2** — Comprensión de los Datos (EDA)
3. **Fase 3** — Preparación de los Datos (Feature Engineering)
4. **Fase 4** — Modelado
5. **Fase 5** — Evaluación
6. **Fase 6** — Despliegue (exportación para Streamlit)

## ⚙️ Configuración del Entorno

In [ ]:
# Instalar dependencias necesarias en Colab
!pip install scikit-learn imbalanced-learn xgboost openpyxl joblib -q
print("✅ Dependencias instaladas")

In [ ]:
# Importar todas las librerías del proyecto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, classification_report, roc_auc_score, roc_curve)
from imblearn.over_sampling import SMOTE
import joblib
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_style("whitegrid")
print("✅ Librerías importadas")

In [ ]:
# Subir y cargar el archivo Excel con el record estudiantil
from google.colab import files
print("📂 Selecciona el archivo REPORTE_RECORD_ESTUDIANTIL_ANONIMIZADO.xlsx")
uploaded = files.upload()
FILENAME = list(uploaded.keys())[0]
df_raw = pd.read_excel(FILENAME)
print(f"✅ Cargado: {df_raw.shape[0]:,} filas × {df_raw.shape[1]} columnas | {df_raw['ESTUDIANTE'].nunique()} estudiantes")

---
# FASE 1 — Comprensión del Negocio

## Contexto
La deserción estudiantil es uno de los principales desafíos de las instituciones de educación superior.
Identificar tempranamente a estudiantes en riesgo permite implementar estrategias de retención.

## Objetivo
Construir un modelo de clasificación binaria que prediga qué estudiantes tienen mayor probabilidad de desertar.

## Definición operativa de Deserción
> Un estudiante se clasifica como **DESERTOR (1)** si su último período de matrícula registrado es **anterior** al ciclo académico más reciente (2025-2026).
> Si aparece en al menos un período de 2025-2026 → **ACTIVO (0)**.

## Criterios de Éxito
| Métrica | Umbral | Justificación |
|---------|--------|---------------|
| **Recall** | > 0.60 | Prioridad: minimizar falsos negativos (desertores no detectados) |
| **F1-Score** | > 0.60 | Equilibrio precisión/recall |
| **Accuracy** | > 0.70 | Rendimiento general aceptable |

---
# FASE 2 — Comprensión de los Datos (EDA)

In [ ]:
# Copia de trabajo, inspección de estructura y tipos de datos
df = df_raw.copy()
print("ESTRUCTURA DEL DATASET")
print("=" * 60)
print(f"Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas\n")
print("Tipos de datos y valores únicos:")
for col in df.columns:
    n = df[col].nunique()
    nulos = df[col].isnull().sum()
    extra = f"  →  {sorted(df[col].dropna().unique().tolist())}" if n <= 12 else ""
    nul_txt = f"  ⚠ {nulos} nulos" if nulos > 0 else ""
    print(f"  {col:25s} [{str(df[col].dtype):8s}] {n:5d} únicos{nul_txt}{extra}")

In [ ]:
# Limpieza: convertir PROMEDIO de texto con coma a float, y eliminar columnas constantes
df['PROMEDIO'] = df['PROMEDIO'].str.replace(',', '.').astype(float)
cols_const = [c for c in df.columns if df[c].nunique() == 1]
print(f"Columnas constantes eliminadas: {cols_const}")
df = df.drop(columns=cols_const)

print(f"\nRangos de variables numéricas:")
print(f"  PROMEDIO:   [{df['PROMEDIO'].min():.1f}, {df['PROMEDIO'].max():.1f}]  (escala 0-10)")
print(f"  ASISTENCIA: [{df['ASISTENCIA'].min()}, {df['ASISTENCIA'].max()}]  (escala 0-100%)")
print(f"  NO. VEZ:    [{df['NO. VEZ'].min()}, {df['NO. VEZ'].max()}]")
print(f"  NIVEL:      [{df['NIVEL'].min()}, {df['NIVEL'].max()}]")

df[['PROMEDIO', 'ASISTENCIA', 'NO. VEZ', 'NIVEL']].describe().round(2)

In [ ]:
# Definir orden cronológico de períodos y crear variable objetivo DESERTOR
PERIODOS = [
    '2023 - 2024 CII', '2023 - 2024 ING2B',
    '2024 - 2025 CI', '2024 - 2025 ING1B', '2024 - 2025 CII', '2024 - 2025 ING2B',
    '2025 - 2026 CI', '2025 - 2026 ING1A', '2025 - 2026 ING1B'
]
ULTIMOS = PERIODOS[6:]  # períodos del ciclo 2025-2026
p2idx = {p: i for i, p in enumerate(PERIODOS)}
idx2p = {i: p for p, i in p2idx.items()}
df['PERIODO_IDX'] = df['PERIODO'].map(p2idx)

# Calcular primer y último período de cada estudiante
est = df.groupby('ESTUDIANTE').agg(
    ULTIMO_IDX=('PERIODO_IDX', 'max'),
    PRIMER_IDX=('PERIODO_IDX', 'min')
).reset_index()
IDX_CORTE = p2idx['2025 - 2026 CI']
est['DESERTOR'] = (est['ULTIMO_IDX'] < IDX_CORTE).astype(int)
est['ULTIMO_PERIODO'] = est['ULTIMO_IDX'].map(idx2p)
df = df.merge(est[['ESTUDIANTE', 'DESERTOR']], on='ESTUDIANTE')

# Mostrar distribución
total = len(est)
n_des = est['DESERTOR'].sum()
n_act = total - n_des
print("VARIABLE OBJETIVO")
print("=" * 40)
print(f"  Activos (0):    {n_act} ({n_act/total*100:.1f}%)")
print(f"  Desertores (1): {n_des} ({n_des/total*100:.1f}%)")
print(f"\nÚltimo período de desertores:")
print(est[est['DESERTOR']==1]['ULTIMO_PERIODO'].value_counts().to_string())

### Visualizaciones del EDA

In [ ]:
# Gráfico 1 — Estudiantes por período académico
fig, ax = plt.subplots(figsize=(14, 5))
ep = df.groupby('PERIODO')['ESTUDIANTE'].nunique().reindex(PERIODOS)
colores = ['#2ecc71' if p in ULTIMOS else '#3498db' for p in PERIODOS]
bars = ax.bar(range(len(PERIODOS)), ep.values, color=colores, edgecolor='white')
ax.set_xticks(range(len(PERIODOS)))
ax.set_xticklabels([p.replace('2023 - 2024','23-24').replace('2024 - 2025','24-25').replace('2025 - 2026','25-26') for p in PERIODOS], fontsize=9)
for b, v in zip(bars, ep.values):
    ax.text(b.get_x()+b.get_width()/2, v+2, str(v), ha='center', fontweight='bold')
ax.set_title('Estudiantes por Período Académico', fontweight='bold')
ax.set_ylabel('Estudiantes')
ax.legend(handles=[plt.Rectangle((0,0),1,1,fc='#3498db'), plt.Rectangle((0,0),1,1,fc='#2ecc71')],
          labels=['Anteriores', 'Recientes (activos)'])
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 2 — Distribución de la variable objetivo (pie + barras)
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
axes[0].pie([n_act, n_des], labels=['No Desertor', 'Desertor'], colors=['#2ecc71','#e74c3c'],
            autopct='%1.1f%%', startangle=90, explode=(0, 0.05), textprops={'fontsize': 12})
axes[0].set_title('Distribución de Deserción', fontweight='bold')

bars = axes[1].bar(['No Desertor','Desertor'], [n_act, n_des], color=['#2ecc71','#e74c3c'], width=0.5, edgecolor='white')
for b, v in zip(bars, [n_act, n_des]):
    axes[1].text(b.get_x()+b.get_width()/2, v+3, f'{v} ({v/total*100:.1f}%)', ha='center', fontweight='bold')
axes[1].set_title('Cantidad por Clase', fontweight='bold')
axes[1].set_ylabel('Estudiantes')
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 3 — Distribución de promedios (histograma + boxplot por estado)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['PROMEDIO'], bins=40, color='#3498db', edgecolor='white', alpha=0.8)
axes[0].axvline(df['PROMEDIO'].mean(), color='red', ls='--', label=f'Media: {df["PROMEDIO"].mean():.2f}')
axes[0].axvline(7.0, color='orange', ls='--', label='Mín. aprobación (7.0)')
axes[0].set_title('Distribución de Promedios', fontweight='bold')
axes[0].set_xlabel('Promedio'); axes[0].legend()

bp = axes[1].boxplot([df[df['ESTADO']=='APROBADA']['PROMEDIO'], df[df['ESTADO']=='REPROBADA']['PROMEDIO']],
                      labels=['Aprobada','Reprobada'], patch_artist=True, widths=0.4)
bp['boxes'][0].set_facecolor('#2ecc71'); bp['boxes'][1].set_facecolor('#e74c3c')
axes[1].set_title('Promedios por Estado', fontweight='bold')
axes[1].set_ylabel('Promedio')
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 4 — Distribución de asistencia y boxplot por deserción
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['ASISTENCIA'], bins=30, color='#9b59b6', edgecolor='white', alpha=0.8)
axes[0].axvline(df['ASISTENCIA'].mean(), color='red', ls='--', label=f'Media: {df["ASISTENCIA"].mean():.1f}%')
axes[0].set_title('Distribución de Asistencia', fontweight='bold')
axes[0].set_xlabel('% Asistencia'); axes[0].legend()

bp = axes[1].boxplot([df[df['DESERTOR']==0]['ASISTENCIA'], df[df['DESERTOR']==1]['ASISTENCIA']],
                      labels=['No Desertor','Desertor'], patch_artist=True, widths=0.4)
bp['boxes'][0].set_facecolor('#2ecc71'); bp['boxes'][1].set_facecolor('#e74c3c')
axes[1].set_title('Asistencia por Condición de Deserción', fontweight='bold')
axes[1].set_ylabel('% Asistencia')
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 5 — Tasa de reprobación por materia
tasa_rep = df.groupby('MATERIA')['ESTADO'].apply(lambda x: (x=='REPROBADA').sum()/len(x)*100).sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(12, 9))
colores = ['#e74c3c' if t > 25 else '#f39c12' if t > 15 else '#2ecc71' for t in tasa_rep.values]
ax.barh(range(len(tasa_rep)), tasa_rep.values, color=colores, edgecolor='white')
ax.set_yticks(range(len(tasa_rep)))
ax.set_yticklabels(tasa_rep.index, fontsize=8)
ax.set_xlabel('Tasa de Reprobación (%)')
ax.set_title('Tasa de Reprobación por Materia', fontweight='bold')
for b, v in zip(ax.patches, tasa_rep.values):
    ax.text(v + 0.3, b.get_y() + b.get_height()/2, f'{v:.1f}%', va='center', fontsize=7)
ax.legend(handles=[plt.Rectangle((0,0),1,1,fc=c) for c in ['#e74c3c','#f39c12','#2ecc71']],
          labels=['>25% (alta)', '15-25% (media)', '<15% (baja)'], loc='lower right')
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 6 — Promedios: histograma superpuesto desertores vs activos
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(df[df['DESERTOR']==0]['PROMEDIO'], bins=30, alpha=0.6, color='#2ecc71', label='No Desertor', density=True)
ax.hist(df[df['DESERTOR']==1]['PROMEDIO'], bins=30, alpha=0.6, color='#e74c3c', label='Desertor', density=True)
ax.set_title('Distribución de Promedios: Desertores vs Activos', fontweight='bold')
ax.set_xlabel('Promedio'); ax.set_ylabel('Densidad'); ax.legend()
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 7 — Aprobación/Reprobación agrupado por deserción
fig, ax = plt.subplots(figsize=(8, 5))
trd = df.groupby(['DESERTOR','ESTADO']).size().unstack(fill_value=0)
trd_pct = trd.div(trd.sum(axis=1), axis=0) * 100
trd_pct.plot(kind='bar', stacked=True, ax=ax, color=['#2ecc71','#e74c3c'], edgecolor='white', width=0.4)
ax.set_xticklabels(['No Desertor', 'Desertor'], rotation=0)
ax.set_title('Aprobación/Reprobación por Deserción', fontweight='bold')
ax.set_ylabel('%'); ax.legend(['Aprobada', 'Reprobada'])
for i, (_, row) in enumerate(trd_pct.iterrows()):
    cum = 0
    for col in trd_pct.columns:
        ax.text(i, cum + row[col]/2, f'{row[col]:.1f}%', ha='center', va='center', fontweight='bold', color='white')
        cum += row[col]
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 8 — Tasa de deserción por nivel máximo alcanzado
niv = df.groupby('ESTUDIANTE').agg(NIVEL_MAX=('NIVEL','max'), DESERTOR=('DESERTOR','first')).reset_index()
tdn = niv.groupby('NIVEL_MAX')['DESERTOR'].mean() * 100

fig, ax = plt.subplots(figsize=(8, 5))
colores_n = ['#e74c3c' if v > 30 else '#f39c12' if v > 15 else '#2ecc71' for v in tdn.values]
ax.bar(tdn.index.astype(str), tdn.values, color=colores_n, edgecolor='white')
for i, v in enumerate(tdn.values):
    ax.text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')
ax.set_title('Tasa de Deserción por Nivel Máximo Alcanzado', fontweight='bold')
ax.set_xlabel('Nivel Máximo'); ax.set_ylabel('Tasa de Deserción (%)')
plt.tight_layout(); plt.show()

---
# FASE 3 — Preparación de los Datos

## Transformación
Se transforma el dataset granular **(4,448 filas: 1 materia × período)** a un dataset de modelado **(488 filas: 1 por estudiante)** creando features agregados que resumen el comportamiento académico.

## Categorías de features:
| Categoría | Qué mide | Ejemplos |
|-----------|----------|----------|
| Rendimiento | Calificaciones | promedio_general, promedio_max, promedio_std |
| Asistencia | Presencia en clase | asistencia_promedio, asistencia_min |
| Aprobación | Éxito académico | tasa_reprobacion, materias_aprobadas |
| Repetición | Materias recursadas | max_vez_cursada, materias_repetidas |
| Progreso | Avance en la carrera | nivel_max, num_periodos_regulares |
| Alertas | Señales de riesgo | materias_nota_cero, pct_asist_cero |
| Contexto | Info complementaria | registros_especiales, carga académica |
| Tendencia | Evolución temporal | cambio_promedio, cambio_asistencia |

In [ ]:
# Identificar registros especiales (movilidad, homologación)
especiales = df[df['GRUPO/PARALELO'].str.contains('MOVILIDAD|HOMOLOGACION|CONVALIDACION', case=False, na=False)]
print(f"Registros especiales: {len(especiales)} ({especiales['ESTUDIANTE'].nunique()} estudiantes)")

# ========================================================================
# BLOQUE DE FEATURE ENGINEERING COMPLETO
# Se agregan todas las métricas a nivel de estudiante en un solo paso
# ========================================================================

# (1) Rendimiento académico — estadísticas de calificaciones y asistencia
feat = df.groupby('ESTUDIANTE').agg(
    promedio_general   = ('PROMEDIO', 'mean'),
    promedio_mediana   = ('PROMEDIO', 'median'),
    promedio_min       = ('PROMEDIO', 'min'),
    promedio_max       = ('PROMEDIO', 'max'),
    promedio_std       = ('PROMEDIO', 'std'),
    asistencia_promedio= ('ASISTENCIA', 'mean'),
    asistencia_mediana = ('ASISTENCIA', 'median'),
    asistencia_min     = ('ASISTENCIA', 'min'),
    asistencia_max     = ('ASISTENCIA', 'max'),
    asistencia_std     = ('ASISTENCIA', 'std'),
).reset_index()
feat[['promedio_std','asistencia_std']] = feat[['promedio_std','asistencia_std']].fillna(0)

# (2) Aprobación y reprobación
apr = df.groupby('ESTUDIANTE').agg(
    total_materias      = ('ESTADO', 'count'),
    materias_aprobadas  = ('ESTADO', lambda x: (x == 'APROBADA').sum()),
    materias_reprobadas = ('ESTADO', lambda x: (x == 'REPROBADA').sum()),
).reset_index()
apr['tasa_aprobacion']  = apr['materias_aprobadas']  / apr['total_materias']
apr['tasa_reprobacion'] = apr['materias_reprobadas'] / apr['total_materias']
feat = feat.merge(apr, on='ESTUDIANTE')

# (3) Repetición de materias
rep = df.groupby('ESTUDIANTE').agg(max_vez_cursada=('NO. VEZ','max'), promedio_vez_cursada=('NO. VEZ','mean')).reset_index()
m_rep = df[df['NO. VEZ'] > 1].groupby('ESTUDIANTE').size().reset_index(name='materias_repetidas')
rep = rep.merge(m_rep, on='ESTUDIANTE', how='left')
rep['materias_repetidas'] = rep['materias_repetidas'].fillna(0).astype(int)
feat = feat.merge(rep, on='ESTUDIANTE')

# (4) Progreso académico — niveles, períodos, materias distintas
prog = df.groupby('ESTUDIANTE').agg(
    nivel_max=('NIVEL','max'), nivel_min=('NIVEL','min'),
    num_periodos=('PERIODO','nunique'), num_materias_distintas=('MATERIA','nunique'),
).reset_index()
prog['avance_niveles'] = prog['nivel_max'] - prog['nivel_min']
preg = df[~df['PERIODO'].str.contains('ING')].groupby('ESTUDIANTE')['PERIODO'].nunique().reset_index(name='num_periodos_regulares')
prog = prog.merge(preg, on='ESTUDIANTE', how='left')
prog['num_periodos_regulares'] = prog['num_periodos_regulares'].fillna(0).astype(int)
feat = feat.merge(prog, on='ESTUDIANTE')

# (5) Señales de alerta — materias con nota/asistencia cero
alert = df.groupby('ESTUDIANTE').agg(
    materias_nota_cero    = ('PROMEDIO',    lambda x: (x == 0).sum()),
    materias_asist_cero   = ('ASISTENCIA',  lambda x: (x == 0).sum()),
    materias_nota_menor5  = ('PROMEDIO',    lambda x: (x < 5).sum()),
    materias_asist_menor50= ('ASISTENCIA',  lambda x: (x < 50).sum()),
).reset_index()
alert = alert.merge(apr[['ESTUDIANTE','total_materias']], on='ESTUDIANTE')
alert['pct_nota_cero']  = alert['materias_nota_cero']  / alert['total_materias']
alert['pct_asist_cero'] = alert['materias_asist_cero'] / alert['total_materias']
alert.drop(columns='total_materias', inplace=True)
feat = feat.merge(alert, on='ESTUDIANTE')

# (6) Contexto — inglés, movilidad, carga académica por período
ing = df[df['MATERIA'].str.contains('INGLÉS', na=False)].groupby('ESTUDIANTE').size().reset_index(name='materias_ingles')
esp = especiales.groupby('ESTUDIANTE').size().reset_index(name='registros_especiales') if len(especiales) > 0 else pd.DataFrame(columns=['ESTUDIANTE','registros_especiales'])
carga = df.groupby(['ESTUDIANTE','PERIODO']).size().reset_index(name='n')
carga = carga.groupby('ESTUDIANTE')['n'].agg(materias_prom_periodo='mean', materias_max_periodo='max').reset_index()
feat = feat.merge(ing, on='ESTUDIANTE', how='left')
feat = feat.merge(esp, on='ESTUDIANTE', how='left')
feat = feat.merge(carga, on='ESTUDIANTE')
feat[['materias_ingles','registros_especiales']] = feat[['materias_ingles','registros_especiales']].fillna(0).astype(int)

# (7) Tendencia temporal — rendimiento en primer vs último período
est_r = est.rename(columns={'PRIMER_IDX':'P','ULTIMO_IDX':'U'})
t1 = df.merge(est_r[['ESTUDIANTE','P']], on='ESTUDIANTE')
t1 = t1[t1['PERIODO_IDX'] == t1['P']]
pp = t1.groupby('ESTUDIANTE').agg(prom_primer=('PROMEDIO','mean'), asist_primer=('ASISTENCIA','mean')).reset_index()
t2 = df.merge(est_r[['ESTUDIANTE','U']], on='ESTUDIANTE')
t2 = t2[t2['PERIODO_IDX'] == t2['U']]
pu = t2.groupby('ESTUDIANTE').agg(prom_ultimo=('PROMEDIO','mean'), asist_ultimo=('ASISTENCIA','mean')).reset_index()
tend = pp.merge(pu, on='ESTUDIANTE')
tend['cambio_promedio']    = tend['prom_ultimo']  - tend['prom_primer']
tend['cambio_asistencia']  = tend['asist_ultimo'] - tend['asist_primer']
feat = feat.merge(tend, on='ESTUDIANTE')

# Agregar variable objetivo
feat = feat.merge(est[['ESTUDIANTE','DESERTOR']], on='ESTUDIANTE')
feat = feat.fillna(0)

print(f"✅ Dataset de modelado creado: {feat.shape[0]} estudiantes × {feat.shape[1]} columnas")
print(f"   Features creados: {feat.shape[1] - 2} (excluyendo ESTUDIANTE y DESERTOR)")
feat.head()

### Selección de Features

In [ ]:
# Selección de features combinando correlación de Pearson + información mutua,
# eliminando variables redundantes (correlación inter-features > 0.90)

feature_cols = [c for c in feat.columns if c not in ['ESTUDIANTE','DESERTOR']]
X_all, y_all = feat[feature_cols], feat['DESERTOR']

# Correlación con la variable objetivo
corr = X_all.corrwith(y_all).abs()

# Información mutua (captura relaciones no lineales)
mi = pd.Series(mutual_info_classif(X_all, y_all, random_state=42), index=feature_cols)

# Score combinado normalizado
cn = (corr - corr.min()) / (corr.max() - corr.min())
mn = (mi - mi.min()) / (mi.max() - mi.min())
score = ((cn + mn) / 2).sort_values(ascending=False)

# Detectar pares de features con alta correlación entre sí (multicolinealidad)
corr_mat = X_all.corr().abs()
upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(bool))
high_pairs = [(c1, c2) for c1 in upper.columns for c2 in upper.index if pd.notna(upper.loc[c2, c1]) and upper.loc[c2, c1] > 0.90]

# Seleccionar features de mayor a menor score, descartando los redundantes
FEATURES = []
descartados = set()
for f in score.index:
    if f in descartados:
        continue
    FEATURES.append(f)
    for a, b in high_pairs:
        if f == a and b not in FEATURES: descartados.add(b)
        elif f == b and a not in FEATURES: descartados.add(a)
FEATURES = [f for f in FEATURES if score[f] > 0.05]

print(f"FEATURES SELECCIONADOS: {len(FEATURES)}")
print("=" * 60)
for i, f in enumerate(FEATURES, 1):
    print(f"  {i:2d}. {f:30s} corr={X_all[f].corr(y_all):+.3f}  MI={mi[f]:.3f}")
print(f"\nDescartados por redundancia: {len(descartados)}")

In [ ]:
# Gráfico 9 — Correlación de features seleccionados con DESERTOR
fig, ax = plt.subplots(figsize=(10, 7))
corr_deser = X_all[FEATURES].corrwith(y_all).sort_values()
colores = ['#e74c3c' if v > 0 else '#2ecc71' for v in corr_deser.values]
ax.barh(range(len(corr_deser)), corr_deser.values, color=colores, edgecolor='white')
ax.set_yticks(range(len(corr_deser)))
ax.set_yticklabels(corr_deser.index, fontsize=8)
ax.set_xlabel('Correlación con DESERTOR')
ax.set_title('Features Seleccionados — Correlación con Deserción', fontweight='bold')
ax.axvline(0, color='black', lw=0.5)
for i, v in enumerate(corr_deser.values):
    ax.text(v + (0.01 if v >= 0 else -0.01), i, f'{v:.3f}',
            va='center', ha='left' if v >= 0 else 'right', fontsize=7, fontweight='bold')
plt.tight_layout(); plt.show()

In [ ]:
# División train/test estratificada y escalado
X = feat[FEATURES].copy()
y = feat['DESERTOR'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_sc = pd.DataFrame(scaler.fit_transform(X_train), columns=FEATURES, index=X_train.index)
X_test_sc  = pd.DataFrame(scaler.transform(X_test),      columns=FEATURES, index=X_test.index)

# SMOTE para balancear clases en entrenamiento
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_sc, y_train)

print(f"Train original:  {len(X_train)} (activos={int((y_train==0).sum())}, desertores={int((y_train==1).sum())})")
print(f"Train SMOTE:     {len(X_train_bal)} (activos={int((y_train_bal==0).sum())}, desertores={int((y_train_bal==1).sum())})")
print(f"Test:            {len(X_test)} (activos={int((y_test==0).sum())}, desertores={int((y_test==1).sum())})")

---
# FASE 4 — Modelado

Se entrenan y comparan **4 algoritmos** de clasificación:

| Modelo | Tipo | Por qué se eligió |
|--------|------|-------------------|
| Regresión Logística | Lineal | Baseline interpretable |
| Random Forest | Ensemble (bagging) | Robusto, maneja no linealidades |
| Gradient Boosting | Ensemble (boosting) | Alta precisión en tabulares |
| SVM | Kernel | Buen rendimiento en dimensiones moderadas |

Todos se evalúan con **validación cruzada 5-fold** y se prueban en el **test set** (20%).

In [ ]:
# Entrenar 4 modelos, evaluar con validación cruzada y predecir en test
modelos = {
    'Regresión Logística': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    'Random Forest':       RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    'Gradient Boosting':   GradientBoostingClassifier(n_estimators=200, random_state=42),
    'SVM':                 SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42),
}

resultados = {}
predicciones = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for nombre, modelo in modelos.items():
    print(f"\n{'='*55}")
    print(f"  {nombre}")
    print(f"{'='*55}")

    # Validación cruzada en el train balanceado
    f1_cv = cross_val_score(modelo, X_train_bal, y_train_bal, cv=cv, scoring='f1')
    print(f"  F1 CV (5-fold): {f1_cv.mean():.4f} ± {f1_cv.std():.4f}")

    # Entrenar y predecir en test
    modelo.fit(X_train_bal, y_train_bal)
    y_pred = modelo.predict(X_test_sc)
    y_prob = modelo.predict_proba(X_test_sc)[:, 1]

    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec  = recall_score(y_test, y_pred)
    f1   = f1_score(y_test, y_pred)
    auc  = roc_auc_score(y_test, y_prob)

    resultados[nombre] = {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1-Score': f1, 'AUC-ROC': auc, 'F1_CV': f1_cv.mean()}
    predicciones[nombre] = {'y_pred': y_pred, 'y_prob': y_prob, 'modelo': modelo}

    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    print(f"  AUC-ROC:   {auc:.4f}")

---
# FASE 5 — Evaluación

In [ ]:
# Tabla comparativa de modelos, ordenada por F1-Score
df_res = pd.DataFrame(resultados).T.sort_values('F1-Score', ascending=False)
mejor_nombre = df_res.index[0]

print("COMPARACIÓN DE MODELOS")
print("=" * 75)
print(df_res.round(4).to_string())
print(f"\n🏆 Mejor modelo: {mejor_nombre} (F1={df_res.loc[mejor_nombre, 'F1-Score']:.4f})")

In [ ]:
# Gráfico 10 — Comparación de métricas entre modelos (barras agrupadas)
fig, ax = plt.subplots(figsize=(12, 6))
metricas = ['Accuracy','Precision','Recall','F1-Score','AUC-ROC']
x = np.arange(len(metricas))
width = 0.18
colores_m = ['#3498db','#2ecc71','#e67e22','#9b59b6']

for i, (nombre, vals) in enumerate(df_res.iterrows()):
    v = [vals[m] for m in metricas]
    bars = ax.bar(x + i * width, v, width, label=nombre, color=colores_m[i], edgecolor='white')
    for b, val in zip(bars, v):
        ax.text(b.get_x() + b.get_width()/2, val + 0.01, f'{val:.2f}', ha='center', fontsize=7)

ax.set_xticks(x + width * 1.5); ax.set_xticklabels(metricas)
ax.set_ylim(0, 1.15); ax.set_title('Comparación de Modelos', fontweight='bold')
ax.legend(loc='upper right'); ax.set_ylabel('Score')
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 11 — Matrices de confusión de los 4 modelos
fig, axes = plt.subplots(1, 4, figsize=(20, 4))
for ax, (nombre, data) in zip(axes, predicciones.items()):
    cm = confusion_matrix(y_test, data['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, annot_kws={'size': 14},
                xticklabels=['Activo','Desertor'], yticklabels=['Activo','Desertor'])
    ax.set_title(nombre, fontweight='bold', fontsize=10)
    ax.set_ylabel('Real'); ax.set_xlabel('Predicho')
plt.suptitle('Matrices de Confusión', fontweight='bold', fontsize=14, y=1.02)
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 12 — Curvas ROC
fig, ax = plt.subplots(figsize=(8, 6))
colores_roc = ['#3498db','#2ecc71','#e67e22','#9b59b6']
for (nombre, data), color in zip(predicciones.items(), colores_roc):
    fpr, tpr, _ = roc_curve(y_test, data['y_prob'])
    auc_val = roc_auc_score(y_test, data['y_prob'])
    ax.plot(fpr, tpr, color=color, lw=2, label=f'{nombre} (AUC={auc_val:.3f})')
ax.plot([0,1], [0,1], 'k--', lw=1, alpha=0.5)
ax.set_xlabel('Tasa de Falsos Positivos'); ax.set_ylabel('Tasa de Verdaderos Positivos')
ax.set_title('Curvas ROC', fontweight='bold'); ax.legend(loc='lower right')
plt.tight_layout(); plt.show()

In [ ]:
# Gráfico 13 — Importancia de variables del mejor modelo
mejor_mod = predicciones[mejor_nombre]['modelo']

if hasattr(mejor_mod, 'feature_importances_'):
    imp = pd.Series(mejor_mod.feature_importances_, index=FEATURES).sort_values()
elif hasattr(mejor_mod, 'coef_'):
    imp = pd.Series(np.abs(mejor_mod.coef_[0]), index=FEATURES).sort_values()
else:
    imp = mi[FEATURES].sort_values()

fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(range(len(imp)), imp.values, color='#2980b9', edgecolor='white')
ax.set_yticks(range(len(imp))); ax.set_yticklabels(imp.index, fontsize=8)
ax.set_xlabel('Importancia')
ax.set_title(f'Importancia de Variables — {mejor_nombre}', fontweight='bold')
for i, v in enumerate(imp.values):
    ax.text(v + 0.002, i, f'{v:.4f}', va='center', fontsize=7)
plt.tight_layout(); plt.show()

In [ ]:
# Reporte de clasificación detallado del mejor modelo
print(f"REPORTE DE CLASIFICACIÓN — {mejor_nombre}")
print("=" * 55)
print(classification_report(y_test, predicciones[mejor_nombre]['y_pred'],
                            target_names=['Activo (0)', 'Desertor (1)']))

In [ ]:
# Optimización de hiperparámetros del mejor modelo con GridSearchCV
if 'Random Forest' in mejor_nombre:
    param_grid = {'n_estimators':[100,200,300], 'max_depth':[5,10,15,None], 'min_samples_split':[2,5,10]}
    gs = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42),
                      param_grid, cv=5, scoring='f1', n_jobs=-1)
elif 'Gradient' in mejor_nombre:
    param_grid = {'n_estimators':[100,200,300], 'max_depth':[3,5,7], 'learning_rate':[0.05,0.1,0.2]}
    gs = GridSearchCV(GradientBoostingClassifier(random_state=42),
                      param_grid, cv=5, scoring='f1', n_jobs=-1)
elif 'SVM' in mejor_nombre:
    param_grid = {'C':[0.1,1,10], 'gamma':['scale','auto'], 'kernel':['rbf','poly']}
    gs = GridSearchCV(SVC(class_weight='balanced', probability=True, random_state=42),
                      param_grid, cv=5, scoring='f1', n_jobs=-1)
else:
    param_grid = {'C':[0.01,0.1,1,10], 'penalty':['l1','l2'], 'solver':['saga']}
    gs = GridSearchCV(LogisticRegression(max_iter=2000, class_weight='balanced', random_state=42),
                      param_grid, cv=5, scoring='f1', n_jobs=-1)

gs.fit(X_train_bal, y_train_bal)
modelo_final = gs.best_estimator_
y_pred_opt = modelo_final.predict(X_test_sc)

print(f"Mejores hiperparámetros: {gs.best_params_}")
print(f"F1 optimizado en test: {f1_score(y_test, y_pred_opt):.4f}")
print(f"Recall optimizado:     {recall_score(y_test, y_pred_opt):.4f}")
print(f"Accuracy optimizado:   {accuracy_score(y_test, y_pred_opt):.4f}")

---
# FASE 6 — Despliegue

Se exportan todos los artefactos necesarios para la aplicación Streamlit:
- `modelo_desercion.pkl` — Modelo entrenado optimizado
- `scaler.pkl` — Escalador entrenado
- `features.pkl` — Lista de features en orden
- `resultados_modelos.pkl` — Métricas de todos los modelos
- `dataset_modelado.csv` — Dataset procesado por estudiante
- `feature_importances.csv` — Importancia de variables

In [ ]:
# Exportar artefactos para Streamlit
joblib.dump(modelo_final, 'modelo_desercion.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(FEATURES, 'features.pkl')
joblib.dump(dict(resultados), 'resultados_modelos.pkl')
feat.to_csv('dataset_modelado.csv', index=False)

# Importancias del modelo final
if hasattr(modelo_final, 'feature_importances_'):
    imp_final = pd.Series(modelo_final.feature_importances_, index=FEATURES)
elif hasattr(modelo_final, 'coef_'):
    imp_final = pd.Series(np.abs(modelo_final.coef_[0]), index=FEATURES)
else:
    imp_final = mi[FEATURES]
imp_final.to_csv('feature_importances.csv')

# Guardar y_test y predicciones para reproducibilidad
pd.DataFrame({'y_test': y_test.values, 'y_pred': y_pred_opt}).to_csv('test_predictions.csv', index=False)

print("✅ Artefactos exportados:")
for f in ['modelo_desercion.pkl','scaler.pkl','features.pkl','resultados_modelos.pkl',
          'dataset_modelado.csv','feature_importances.csv','test_predictions.csv']:
    print(f"   📦 {f}")

In [ ]:
# Descargar todos los archivos para usar con Streamlit (solo en Colab)
from google.colab import files
for f in ['modelo_desercion.pkl','scaler.pkl','features.pkl','resultados_modelos.pkl',
          'dataset_modelado.csv','feature_importances.csv','test_predictions.csv']:
    files.download(f)
print("\n✅ Todos los archivos descargados. Colócalos junto a app.py para ejecutar Streamlit.")

---
## Conclusiones

1. Se aplicó la **metodología CRISP-DM** completa sobre el dataset de record estudiantil (4,448 registros, 488 estudiantes).
2. Se definió la variable objetivo **DESERTOR** a partir del último período de matrícula: 208 desertores (42.6%) vs 280 activos (57.4%).
3. Se crearon **40+ features** agregados a nivel de estudiante, seleccionando los más relevantes mediante correlación de Pearson e información mutua, eliminando multicolinealidad.
4. Se evaluaron **4 modelos** de clasificación con validación cruzada 5-fold y SMOTE para balanceo.
5. El mejor modelo fue **optimizado** con GridSearchCV y exportado para la aplicación Streamlit.
6. Los **factores más predictivos** de deserción son: tasa de reprobación, materias aprobadas, promedio académico y asistencia.